# Statistical treatment for PASTIS

<font color='red'>**This notebook is outdated as of 9 May 2021.  
Please use more recent notebooks for help.**</font>

Getting into a full statistical treatment of the WFE requirements both mode-based as well as segmnet-based, using normal distributions and covariance matrices.

1. set target contrat in code cell 2 (e.g. `1e-10`)
2. set apodizer design in code cell 3 (e.g. `small`)
3. comment in correct data path in code cell 3 (e.g. `[...]/2020-01-27T23-57-00_luvoir-small`)

In [ ]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline
from astropy.io import fits
import astropy.units as u
import hcipy
from pastis.e2e_simulators.generic_segmented_telescopes import SegmentedMirror

from pastis.config import CONFIG_PASTIS
import pastis.util as util
from pastis.e2e_simulators.luvoir_imaging import LuvoirAPLC

In [ ]:
nmodes = 120
c_target = 1e-10

## Instantiate LUVOIR telescope for full functionality

In [ ]:
apodizer_design = 'small'

savedpath = '/Users/ilaginja/Documents/data_from_repos/pastis_data/2020-01-27T23-57-00_luvoir-small'
#savedpath = '/Users/ilaginja/Documents/data_from_repos/pastis_data/2020-01-28T02-17-18_luvoir-medium'
#savedpath = '/Users/ilaginja/Documents/data_from_repos/pastis_data/2020-01-28T04-45-55_luvoir-large'

In [ ]:
# Instantiate LUVOIR
sampling = 4
# This path is specific to the paths used in the LuvoirAPLC class
optics_input = os.path.join(util.find_repo_location(), CONFIG_PASTIS.get('LUVOIR', 'optics_path_in_repo'))

luvoir = LuvoirAPLC(optics_input, apodizer_design, sampling)

In [ ]:
# Make reference image
luvoir.flatten()
psf_unaber, ref = luvoir.calc_psf(ref=True)
norm = ref.max()

In [ ]:
# Make dark hole
dh_outer = hcipy.circular_aperture(2*luvoir.apod_dict[apodizer_design]['owa'] * luvoir.lam_over_d)(luvoir.focal_det)
dh_inner = hcipy.circular_aperture(2*luvoir.apod_dict[apodizer_design]['iwa'] * luvoir.lam_over_d)(luvoir.focal_det)
dh_mask = (dh_outer - dh_inner).astype('bool')

plt.figure(figsize=(18, 6))
plt.subplot(131)
hcipy.imshow_field(psf_unaber/norm, norm=LogNorm())
plt.subplot(132)
hcipy.imshow_field(dh_mask)
plt.subplot(133)
hcipy.imshow_field(psf_unaber/norm, norm=LogNorm(), mask=dh_mask)

In [ ]:
dh_intensity = psf_unaber/norm * dh_mask
baseline_contrast = util.dh_mean(dh_intensity, dh_mask)
#np.mean(dh_intensity[np.where(dh_intensity != 0)])
print('Baseline contrast:', baseline_contrast)

In [ ]:
# Load PASTIS modes - piston value per segment per mode
pastismodes = np.loadtxt(os.path.join(savedpath, 'results', 'pastis_modes.txt'))
print('pastismodes.shape: {}'.format(pastismodes.shape))
# pastismodes[segs, modes]

# Load PASTIS matrix
pastismatrix = fits.getdata(os.path.join(savedpath, 'matrix_numerical', 'PASTISmatrix_num_piston_Noll1.fits'))

# Load sigma vector
sigmas = np.loadtxt(os.path.join(savedpath, 'results', 'mode_requirements_1e-10_uniform.txt'))
print(sigmas)

In [ ]:
# Calculate the inverse of the PASTIS mode matrix
# This is ModeToSegs in Mathematica
modestosegs = np.linalg.pinv(pastismodes)
# modestosegs[modes, segs]

## Static sigmas to avg contrast

In [ ]:
# Calculate mean contrast of all modes with PASTIS matrix AND the sigmas, to make sure this works
c_avg_sigma = []
for i in range(nmodes):
    c_avg_sigma.append(util.pastis_contrast(sigmas[i] * pastismodes[:,i]*u.nm, pastismatrix))
    
print(c_avg_sigma)

Confirm that all of of them, with the baseline contarst, add up to the target contrast.

In [ ]:
np.sum(c_avg_sigma) + baseline_contrast

Draw random numbers

In [ ]:
# Create x-array
x_vals = np.zeros_like(sigmas)
for i, sig in enumerate(sigmas):
    x_vals[i] = np.random.normal(loc=0, scale=sig)

print(x_vals.shape)
print(x_vals)

In [ ]:
# Calculate mean contrast of all modes with PASTIS matrix AND the sigmas, to make sure this works
c_avg_sigma = []
for i in range(nmodes):
    c_avg_sigma.append(util.pastis_contrast(x_vals[i] * pastismodes[:,i]*u.nm, pastismatrix))
    
print(c_avg_sigma)

In [ ]:
np.sum(c_avg_sigma) + baseline_contrast

Loop it up - cumulatively

In [ ]:
runnum = 1000
xs_list = []

for l in range(runnum):
    x_vals = np.zeros_like(sigmas)
    for i, sig in enumerate(sigmas):
        x_vals[i] = np.random.normal(loc=0, scale=sig)
        #x_vals[i] = np.random.uniform(-sig, sig)

    xs_list.append(x_vals)
xs_list = np.array(xs_list)
xs_list.shape

In [ ]:
c_avg_list = []
for a in range(runnum):

    c_avg_sigma = []
    for i in range(nmodes):
        c_avg_sigma.append(util.pastis_contrast(xs_list[a][i] * pastismodes[:,i]*u.nm, pastismatrix))

    c_avg_list.append(np.sum(c_avg_sigma) + baseline_contrast)

In [ ]:
print(np.mean(c_avg_list))
print(np.std(c_avg_list))

In [ ]:
np.array(c_avg_list)

Which results in the input target contrast.

Same thing for a single mode at a time

In [ ]:
modechoice = 23
runnum = 1000
xs_list = []

for l in range(runnum):
    x_vals = np.zeros_like(sigmas)
    x_vals[modechoice] = np.random.normal(loc=0, scale=sigmas[modechoice])
    xs_list.append(x_vals)
    
xs_list = np.array(xs_list)
xs_list.shape

In [ ]:
c_avg_list = []
for a in range(runnum):

    c_avg_sigma = util.pastis_contrast(xs_list[a][modechoice] * pastismodes[:,i]*u.nm, pastismatrix)
    c_avg_list.append(c_avg_sigma + baseline_contrast)

In [ ]:
print(np.mean(c_avg_list))
print(np.std(c_avg_list))

Which is equivalent to:

In [ ]:
c_target/120 + baseline_contrast

We are able to define a static sigma value from the target contrast and singular values (See equation for sigmas). With this static sigmas, you get the correct contrast in a deterministic sense, see cumulative contrast plot.

We have verified in this notebook that a Gaussian distribution with a zero mean and std = sigma prodices an average (statistical) DH mean (spatial) contrast. It's the statistical average of the mean contrast. We have checked this both for individual modes as well as the ensemble of modes all toghether.

## Segment requirements

Now we want to transform this into segment-based requirements/tolerances/constraints $\mu$.

We now want to verify that this works in segment space (as opposed to the mode-base the sigmas are defined in).

We're doing $ y = M \cdot x$ here.

In [ ]:
runnum = 1000
ys_list = []

for l in range(runnum):
    x_vals = np.zeros_like(sigmas)
    for i, sig in enumerate(sigmas):
        x_vals[i] = np.random.normal(loc=0, scale=sig)
    
    # Calculate y vector
    y_vals = np.dot(pastismodes, x_vals)
        
    ys_list.append(y_vals)
ys_list = np.array(ys_list)
ys_list.shape

In [ ]:
c_avg_list = []
for a in range(runnum):

    c_avg_sigma = util.pastis_contrast(ys_list[a]*u.nm, pastismatrix)
    #c_avg_list.append(c_avg_sigma + baseline_contrast)
    c_avg_list.append(c_avg_sigma)

In [ ]:
ys_list

In [ ]:
print(np.mean(c_avg_list))
print(np.std(c_avg_list))

This means it is completely equivalent to work in the mode basis or in the segment basis.

We have now a bunch of ys for which this works. Now we want to figure out what distributions these y maps (correct realizations of $\mu$s follow, and what we can quote as segment requirements.
The y capture everything, including the cross-terms of the covariance matrix. Just averaging the ys will probably not be enough, but we can run them through the MC and see. If the off-diagonal terms of the Covariance matrix are not large though, this will be very similar.

We need to assemble the Covariance matrix for y, $C_y$.

## Covariance matrix $C_y$

Build $C_x$ by hand by dumping the square of the std (= variance) into the diagonal of a properly sized matrix.

In [ ]:
cx = np.diag(np.square(sigmas))
#print(cx)

$$y = M \cdot x$$
$$C_x = E(x \cdot x^T)$$
$$C_y = E(y \cdot y^T) = M \cdot E(x \cdot x^T) \cdot M^T$$
$$C_y = M \cdot C_x \cdot M^T$$

In [ ]:
cy = np.dot(pastismodes, np.dot(cx, np.transpose(pastismodes)))

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(np.log10(cx))
plt.xlabel('modes')
plt.ylabel('modes')
plt.colorbar()

In [ ]:
np.diag(cx)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(cy)
plt.xlabel('segment')
plt.ylabel('segment')
plt.colorbar()

In [ ]:
cy[0,0]

In [ ]:
testmap = np.sqrt(np.diag(cy))
print(testmap)

In [ ]:
luvoir.flatten()
testmap *= u.nm
for seg, mu in enumerate(testmap):
    luvoir.set_segment(seg+1, (mu).to(u.m).value/2, 0, 0)
psf, ref = luvoir.calc_psf(ref=True, display_intermediate=True)

In [ ]:
testmap

In [ ]:
# Recalculate the xs
runnum = 1000
xs_list = []

for l in range(runnum):
    x_vals = np.zeros_like(sigmas)
    for i, sig in enumerate(sigmas):
        x_vals[i] = np.random.normal(loc=0, scale=sig) #sig
        #x_vals[i] = np.random.uniform(-sig, sig)

    xs_list.append(x_vals)
xs_list = np.array(xs_list)
xs_list.shape

In [ ]:
# Empirical Cx
# !! Make sure to rerun the correct xs_list
pall = []
for i in range(runnum):
    pall.append(np.outer(xs_list[i], np.transpose(xs_list[i])))

pall = np.array(pall)

In [ ]:
cx_emp = np.mean(pall, axis=0)
print(cx_emp.shape)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(np.log10(cx_emp))
plt.colorbar()

In [ ]:
np.sqrt(np.diag(cx_emp))   # these should be equivlaneet to the sigmas

In [ ]:
plt.plot(sigmas)
plt.semilogy()

In [ ]:
sigmas

In [ ]:
np.allclose(cx, cx_emp)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(np.log10(cx))
plt.colorbar()

In [ ]:
np.sqrt(np.diag(cx))

In [ ]:
np.sqrt(np.diag(cy))

In [ ]:
# Empirical Cy
pall_y = []
for i in range(ys_list.shape[0]):
    pall_y.append(np.outer(ys_list[i], np.transpose(ys_list[i])))

print('runnum = {}'.format(ys_list.shape[0]))
pall_y = np.array(pall_y)

In [ ]:
cy_emp = np.mean(pall_y, axis=0)
print(cy_emp.shape)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(cy_emp)
plt.colorbar()

Get mean of the ys directly, ignoring the Covariance matrix.

In [ ]:
y_direkt_mean = np.mean(ys_list, axis=0)
print(y_direkt_mean.shape)
print(y_direkt_mean)

In [ ]:
np.mean(y_direkt_mean)

In [ ]:
# Put those mean ys/mus on the simulator
luvoir.flatten()
y_direkt_mean *= u.nm
for seg, mu in enumerate(y_direkt_mean):
    luvoir.set_segment(seg+1, (mu).to(u.m).value/2, 0, 0)
psf, ref = luvoir.calc_psf(ref=True, display_intermediate=True)

In [ ]:
avg_con = util.pastis_contrast(y_direkt_mean, pastismatrix) + baseline_contrast
print(avg_con)

### Some tests

Use an x vector with only one entry (=1) and see whether $y = M \cdot x$ yields a PASTIS mode.

In [ ]:
testmode = -2
x_test = np.zeros(nmodes)
x_test[testmode] = 1

print(x_test)

In [ ]:
y_test = np.dot(pastismodes, x_test)
y_test *= u.nm

In [ ]:
# Put the resulting y on the simulator
luvoir.flatten()
for seg, coef in enumerate(y_test):
    luvoir.set_segment(seg+1, (coef).to(u.m).value/2, 0, 0)
psf, ref = luvoir.calc_psf(ref=True, display_intermediate=True)